In [4]:
import pygame
import cv2
import mediapipe as mp
import sys
import random
import time

# Inicialización de Mediapipe para la detección de parpadeos
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5)

# Inicialización de Pygame
pygame.init()

# Configuración de la ventana
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Luz Verde, Luz Roja")
clock = pygame.time.Clock()
FPS = 30

# Inicialización de Pygame Mixer
pygame.mixer.init()

# Cargar archivo de audio
audio_file = "Coro.mp3"
try:
    pygame.mixer.music.load(audio_file)
except pygame.error:
    print(f"Error al cargar el archivo de audio: {audio_file}")
    sys.exit()

# Cargar imagen de fondo inicial
try:
    initial_background_img = pygame.image.load('calamar3.png')
    initial_background_img = pygame.transform.scale(initial_background_img, (WIDTH, HEIGHT))
except pygame.error:
    print("Error al cargar la imagen 'calamar3.png'. Asegúrate de que existe en el directorio.")
    sys.exit()

# Cargar imágenes de fondo para los estados de luz
try:
    green_light_img = pygame.image.load('calamar2.png')
    green_light_img = pygame.transform.scale(green_light_img, (WIDTH, HEIGHT))
    red_light_img = pygame.image.load('calamar.png')
    red_light_img = pygame.transform.scale(red_light_img, (WIDTH, HEIGHT))
except pygame.error:
    print("Error al cargar las imágenes de fondo. Asegúrate de que existen en el directorio.")
    sys.exit()

# Cargar mirilla
try:
    crosshair_img = pygame.image.load('mirilla.png')
    crosshair_img = pygame.transform.scale(crosshair_img, (100, 100))
except pygame.error:
    print("Error al cargar la imagen 'mirilla.png'. Asegúrate de que existe en el directorio.")
    sys.exit()

# Cargar sonido de disparo
try:
    gunshot_sound = pygame.mixer.Sound('disparo.mp3')
except pygame.error:
    print("Error al cargar el sonido 'disparo.mp3'. Asegúrate de que existe en el directorio.")
    sys.exit()

# Colores
RED = (255, 0, 0)
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Configuración del juego
state = "countdown"  # Estados: "countdown", "waiting", "red", "green"
time_limit = 6
elapsed_time = 0
blink_count = 0
required_blinks = 10
game_over = False
game_won = False
audio_started = False
show_crosshair = False
crosshair_timer = 0
CROSSHAIR_DURATION = 0.5  # Duración de la animación de la mirilla

# Barra de progreso
progress_bar_width = WIDTH - 100
progress_bar_height = 30
progress = 0

# Captura de video
cap = cv2.VideoCapture(0)

# Función para calcular el aspecto del ojo (EAR)
def calculate_ear(landmarks, eye_points):
    a = ((landmarks[eye_points[1]][1] - landmarks[eye_points[5]][1]) ** 2 +
         (landmarks[eye_points[1]][0] - landmarks[eye_points[5]][0]) ** 2) ** 0.5
    b = ((landmarks[eye_points[2]][1] - landmarks[eye_points[4]][1]) ** 2 +
         (landmarks[eye_points[2]][0] - landmarks[eye_points[4]][0]) ** 2) ** 0.5
    c = ((landmarks[eye_points[0]][1] - landmarks[eye_points[3]][1]) ** 2 +
         (landmarks[eye_points[0]][0] - landmarks[eye_points[3]][0]) ** 2) ** 0.5
    ear = (a + b) / (2.0 * c)
    return ear

# Constante para detectar parpadeos
BLINK_THRESHOLD = 0.2
eye_closed = False

# Bucle principal
running = True
countdown_time = 3  # Tiempo de cuenta atrás
countdown_start = time.time()
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Capturar fotogramas de la cámara
    ret, frame = cap.read()
    if not ret:
        print("Error al acceder a la cámara.")
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    # Procesar detección de parpadeos
    landmarks = []
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for id, landmark in enumerate(face_landmarks.landmark):
                h, w, c = frame.shape
                x, y = int(landmark.x * w), int(landmark.y * h)
                landmarks.append((x, y))

            if landmarks:
                left_eye_points = [362, 385, 387, 263, 373, 380]  # Ojo izquierdo
                right_eye_points = [33, 160, 158, 133, 153, 144]  # Ojo derecho

                left_ear = calculate_ear(landmarks, left_eye_points)
                right_ear = calculate_ear(landmarks, right_eye_points)

                ear = (left_ear + right_ear) / 2.0

                if ear < BLINK_THRESHOLD:
                    if not eye_closed:
                        eye_closed = True
                        if state == "green" and not game_won:
                            blink_count += 1
                            progress = min(progress_bar_width, (blink_count / required_blinks) * progress_bar_width)
                            if progress >= progress_bar_width:
                                game_won = True
                        elif state == "red":
                            show_crosshair = True
                            crosshair_timer = time.time()
                            gunshot_sound.play()
                            game_over = True
                else:
                    eye_closed = False

    # Lógica del juego
    if state == "countdown":
        screen.blit(initial_background_img, (0, 0))  # Fondo inicial
        countdown_elapsed = time.time() - countdown_start
        countdown_number = countdown_time - int(countdown_elapsed)
        if countdown_number > 0:
            countdown_text = pygame.font.SysFont(None, 72).render(str(countdown_number), True, WHITE)
            screen.blit(countdown_text, (WIDTH // 2 - countdown_text.get_width() // 2, HEIGHT // 2))
        else:
            state = "waiting"

    elif state == "waiting" and not game_won:
        screen.blit(initial_background_img, (0, 0))  # Fondo inicial
        state = random.choice(["red", "green"])
        elapsed_time = 0
        if not audio_started:
            pygame.mixer.music.play(-1)  # Reproducir música en bucle
            audio_started = True

    elif state == "green" and not game_won:
        screen.blit(green_light_img, (0, 0))  # Fondo verde
        pygame.draw.circle(screen, GREEN, (WIDTH // 2, 50), 30)  # Círculo verde
        elapsed_time += 1 / FPS
        if elapsed_time >= time_limit:
            state = "waiting"

    elif state == "red" and not game_won:
        screen.blit(red_light_img, (0, 0))  # Fondo rojo
        pygame.draw.circle(screen, RED, (WIDTH // 2, 50), 30)  # Círculo rojo
        elapsed_time += 1 / FPS
        if elapsed_time >= time_limit:
            state = "waiting"

    # Dibujar barra de progreso
    pygame.draw.rect(screen, BLACK, (50, HEIGHT - 100, progress_bar_width, progress_bar_height), 2)
    pygame.draw.rect(screen, GREEN, (50, HEIGHT - 100, progress, progress_bar_height))

    # Mostrar mensaje de Game Over y animación de disparo
    if game_over:
        pygame.mixer.music.stop()  # Detener la música
        if show_crosshair:
            crosshair_position = (WIDTH // 2 - crosshair_img.get_width() // 2, HEIGHT // 2 - 150)
            screen.blit(crosshair_img, crosshair_position)
            if time.time() - crosshair_timer > CROSSHAIR_DURATION:
                show_crosshair = False

        game_over_text = pygame.font.SysFont(None, 72).render("GAME OVER!", True, RED)
        screen.blit(game_over_text, (WIDTH // 2 - game_over_text.get_width() // 2, HEIGHT // 2))
        pygame.display.flip()
        time.sleep(3)
        running = False

    # Mostrar mensaje de Juego Superado
    if game_won:
        pygame.mixer.music.stop()  # Detener la música
        win_text = pygame.font.SysFont(None, 72).render("¡Juego Superado!", True, GREEN)
        screen.blit(win_text, (WIDTH // 2 - win_text.get_width() // 2, HEIGHT // 2))
        pygame.display.flip()
        time.sleep(3)
        running = False

    # Actualizar pantalla
    pygame.display.flip()
    clock.tick(FPS)

# Liberar recursos
cap.release()
pygame.quit()
sys.exit()

SystemExit: 